# Module 2: Data Engineering
## Sprint 2: SQL and Data Scraping
## Storing Scraped Information

## Background 
You did an outstanding job this sprint. Now you know SQL databases, how to make basic data operations. You are familiar with the pros and cons of SQL and should know when to choose a NoSQL database. Data scraping is a technique that you can use to create your own dataset. You are also familiar with the basic concepts of Spark. You should be proud of yourself! For the last lesson of this week, you should put all your learnings into one place to collect, process, and store data. Combining these two might require a bit of planning at first but this is what you will need to do now.

---

## Creating the PostgreSQL database
First, you will need to create a database that you will need to use to store the data you will collect. Follow the steps provided in the second lesson of this sprint. Do not forget to remove all secrets and passwords when committing code to the repository. 

## Scraping the data
This lesson, you will need to scrape and store data collected from an online store. You can choose any website that you like (Vinted, Amazon, eBay, etc.). You will need to select three keywords (for example `dress`, `bike`, `bracelet`) and scrape listings from the selected website. You will need to collect at least 3000 samples for each category and store this information of the listing: `category`, `title`, `price`, `url to item`, `url of image`. 

## Structure of the database
As you will be storing data into the relational database, you will need to create tables. There should be two tables with many to one type of relationship.

---

## Concepts to explore
* Creating relational type database
* Creating PostgreSQL database in Heroku
* Writing SQL queries
* Scraping webpages using Beautiful Soup
* Storing and acquiring data using SQL

## Requirements
You should create two files: one that contains functions related to data scraping and another that is created to enable communication to PostgreSQL database created with Heroku. The actual requirements are these:
- Database should be created using Heroku.
- Required tables should be created. Python code should be provided inside `.py` file.
- Scraping functions should be created. The main scraping function should take two arguments: `number of examples to scrape` and `keyword to search`. The main scraping function should return a Pandas `DataFrame` with the records.
- Scrape the website. Get minimum 3000 samples of each category (keyword)
- Data should be inserted into tables of database hosted by Heroku. Provide screenshots proofing that data sits inside the database.
- Join two tables into one using SQL query and export it to `csv` file. Provide a function that makes this action.


In [1]:
pip install beautifulsoup4

In [2]:
pip install fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13503 sha256=45db01d312053363540a3fcbc6d2dd4bcc6fb1548d0d1b0ddfd77a56ae1e4712
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
from google.colab import files

In [5]:
def reed_scraper(job_name: str, examples_total: int) -> pd.DataFrame:
  '''Scrapes unique reed job listings matching keyword.'''
  results = set()
  ua = UserAgent()
  parsed_job_name = job_name.strip().replace(" ", "-")
  dfcolumn_names = ["position", "pay", "employer", "location", "county", "desc_short", "jobtype", "remote", "joburl", "jobrefID"]
  data_remaining = True
  page = 0


  def newpage_setup() -> BeautifulSoup:
    '''Soup object for new page'''
    header = {'User-Agent':str(ua.random)}
    url = f"https://www.reed.co.uk/jobs/{parsed_job_name}-jobs?pageno={page}&sortby=DisplayDate"
    request = requests.get(url, headers=header)
    chicken_stock = BeautifulSoup(request.content, "html.parser")
    return chicken_stock


  def rip_page() -> [set, bool]:
    '''Extract data from all unique job listings on page.'''
    page_results = set()
    soup = newpage_setup()
    joblistings = soup.find_all('article', class_="job-result")
    last_page = len(joblistings) < 25
    job_index = 0

    while ( ( (len(results) + len(page_results)) < examples_total ) & (job_index < len(joblistings)) ):
      page_results.add(rip_line (joblistings[job_index]) )
      job_index += 1
      
    time.sleep(1.5)
    return [page_results, last_page]

    
  def rip_line(job: 'bs4.element.ResultSet') -> tuple:
    '''Extract data from a single job listing.'''

    position = job.find('a', {"data-gtm": "job_click"}).text

    pay = job.find('li', class_="salary").text

    employer = job.find('a', class_="gtmJobListingPostedBy").text

    location_string_generator = job.find('li', class_="location").stripped_strings
    location = next(location_string_generator)
    county = next(location_string_generator, location)

    desc_short = job.find('div', class_="description").find('p').text

    jobtype = job.find('li', class_="time").text

    remote = not (job.find('li', class_="remote") is None)

    joburl = job.find('a', {"data-gtm": "job_click"}).get('href')
    joburl = "https://www.reed.co.uk" + joburl.split("?", 1)[0]
    jobrefID = int (joburl.split('/')[-1])

 #  spoon_request = requests.get(joburl, headers=header)
 #  soup_spoon = BeautifulSoup(spoon_request.content, "html.parser")
 #  full_desc = soup_spoon.find('span', {"itemprop": "description"}).get_text(separator="|")
 #  employer_img = soup_spoon.find('a', {"data-qa": "recruiterLogoLnk"}).find('img').attrs['data-src']

    rippedline = [
                 position,
                 pay,
                 employer,
                 location,
                 county,
                 desc_short,
                 jobtype,
                 remote,
                 joburl,
                 jobrefID
                 ]
    
    return tuple(rippedline)
  

  while ( (len(results) < examples_total) & (data_remaining) ):
    page_state = rip_page()
    results.update(page_state[0])
    data_remaining = not page_state[1]
    page += 1

  if (not data_remaining) & (len(results) < examples_total): print("No more unique job listings remaining to scrape.")
  
  return pd.DataFrame(results, columns=dfcolumn_names, copy=True)

In [14]:
data_scientist_df = reed_scraper("data scientist", 3000)
data_scientist_df.tail()

No more unique job listings remaining to scrape.


,position,pay,employer,location,county,desc_short,jobtype,remote,joburl,jobrefID
727,Early Detection Research Programme Manager,Competitive salary,The Guardian,North East England,North East England,Early Detection Research Programme ManagerThis...,"Permanent, full-time",True,https://www.reed.co.uk/jobs/early-detection-re...,43460301
728,Formulation Scientist,Salary negotiable,PE Global,Nottingham,Nottinghamshire,PE Global is currently recruiting for a Formul...,"Contract, full-time",False,https://www.reed.co.uk/jobs/formulation-scient...,43203982
729,Finance Analyst Apprentice,"£20,000 per annum, inc benefits",Multiverse,London,London,A new opportunity has arisen in Charles Taylor...,"Permanent, full-time",True,https://www.reed.co.uk/jobs/finance-analyst-ap...,43139610
730,Analytics Manager Data Science,"£50,000 - £60,000 per annum",Cactus Search,The Home,Shropshire,Our Client is looking to source an experienced...,"Permanent, full-time",False,https://www.reed.co.uk/jobs/analytics-manager-...,43270273
731,Business Analyst - Digital Health Start-Up,Salary negotiable,Zest Business Group,London,London,Zest Scientific is partnered with one of the U...,"Contract, full-time",True,https://www.reed.co.uk/jobs/business-analyst-d...,43256002


In [15]:
data_scientist_df.to_csv("data scientist")

## Evaluation criteria
1. The requirements are met (database and tables in it are created, data is scraped and stored)
2. Data scraping functions are written.
3. Code meets expected standards (type hints, PEP8 standards)
4. Documentation is provided (comments are written where needed, README.md file is created)


## Sample correction questions

During a correction, you may get asked questions that test your understanding of covered topics.

- Why do we store information inside external databases? Tell two advantages and disadvantages when comparing local and external storage.
- What is the difference between SQL and NoSQL databases? Tell two examples: one where you would want to use PostgreSQL database and another where you would choose Cassandra.
- How to properly set up a data scraping strategy? What are the key steps that you must make to successfully create a dataset using web scraping technique?
- Why do we need Spark? Tell about one use case where Spark would improve scalability and speed of queries when comparing to traditional relational type database's engine.